# Multi-Agent Debate Demo

This notebook demonstrates the RoundtableAI multi-agent debate system:
- **Natural language query routing** - The orchestrator classifies queries and routes to appropriate agent(s)
- **Risk tolerance inference** - Automatically infers investor risk profile from query language
- **Multi-agent debates** - Running debates with consensus tracking
- **Single agent queries** - Simple factual queries handled by single specialists

The system uses three specialist agents:
1. **Fundamental Agent**: Analyzes financial statements
2. **Sentiment Agent**: Analyzes news sentiment  
3. **Valuation Agent**: Analyzes risk-return metrics

## 1. Setup and Imports

In [ ]:
# Add project root to path
import sys
sys.path.insert(0, '..')

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# Import debate components
from agents import (
    DebateOrchestrator,
    create_debate_orchestrator,
    DebateManager,
    DebateState,
    Recommendation,
    get_llm
)

# Import evaluation components
from evaluation import DebateEvaluator

# Import configuration
from utils.config import get_debate_config

print("Imports successful!")

## 2. Initialize the LLM (Google Gemini)

Connect to Gemini 2.0 Flash via Google AI API.

**Prerequisites:**
1. Get API key from: https://aistudio.google.com/apikey
2. Set `GOOGLE_API_KEY` in `.env` file

In [ ]:
# Connect to Gemini LLM (singleton - reused across agents)
llm = get_llm(
    model_name="gemini-2.0-flash",
    temperature=0.3
)

## 3. Create the Debate Orchestrator

In [ ]:
# View debate configuration
debate_config = get_debate_config()
print("Debate Configuration:")
for key, value in debate_config.items():
    print(f"  {key}: {value}")

In [ ]:
# Callback to track debate progress
def on_message(message):
    """Called when an agent posts a message."""
    agent = message.agent_type
    rec = message.recommendation.value
    conf = message.confidence
    round_num = message.round_number
    print(f"[Round {round_num}] {agent.upper()}: {rec} ({conf:.0%} confidence)")

def on_round_complete(round_result):
    """Called when a debate round completes."""
    round_num = round_result.round_number
    consensus = round_result.consensus_percentage
    leading = round_result.leading_recommendation.value
    print(f"\n--- Round {round_num} Complete ---")
    print(f"Consensus: {consensus:.0%}, Leading: {leading}")
    print()

In [ ]:
# Create the orchestrator
orchestrator = create_debate_orchestrator(
    llm=llm,
    max_rounds=5,
    consensus_threshold=0.75,
    on_message_callback=on_message
)

print("Orchestrator created successfully!")
print(f"Agents: {list(orchestrator.agents.keys())}")

## 4. Route Natural Language Queries

The orchestrator's `route_query()` method:
1. **Classifies the query** - Determines if multi-agent debate or single agent is needed
2. **Extracts company/ticker** - Identifies which stock is being asked about
3. **Infers risk tolerance** - Detects conservative/moderate/aggressive from language
4. **Routes appropriately** - Sends to debate or single agent

In [ ]:
# Example 1: Investment recommendation query (triggers multi-agent debate)
# Risk tolerance will be inferred as "moderate" (default)

user_query = "Should I invest in Maybank? I'm looking for a balanced investment."

print(f"User Query: {user_query}")
print(f"\n{'='*60}")
print("Processing query...")
print(f"{'='*60}\n")

# Route the query - this will:
# 1. Classify the query (needs debate)
# 2. Extract company (Maybank)
# 3. Infer risk tolerance (moderate - "balanced")
# 4. Run multi-agent debate
result = orchestrator.route_query(user_query)

print(f"\nRoute Type: {result['route_type']}")
print(f"Risk Tolerance: {result.get('risk_tolerance', 'N/A')}")
print(f"Classification: {result.get('classification', {})}")

In [ ]:
# Display result based on route type
if result['route_type'] == 'debate':
    print("\n" + "="*60)
    print("MULTI-AGENT DEBATE RESULT")
    print("="*60)
    
    print(f"\nRecommendation: {result['recommendation']}")
    print(f"Confidence: {result['confidence']:.0%}")
    print(f"Consensus Level: {result['consensus']:.0%}")
    print(f"Risk Tolerance Applied: {result['risk_tolerance'].upper()}")
    
    # Access full result for more details
    final_rec = result['full_result']
    
    print("\nKey Points:")
    for point in final_rec.key_points:
        print(f"  - {point}")

    print("\nKey Risks:")
    for risk in final_rec.risks:
        print(f"  - {risk}")
        
elif result['route_type'] == 'single_agent':
    print("\n" + "="*60)
    print("SINGLE AGENT RESPONSE")
    print("="*60)
    print(f"\nAgent Used: {result['agent_used']}")
    print(f"Response:\n{result['response'][:1000]}...")
    
else:
    print(f"\nError: {result['response']}")

In [ ]:
# View agent breakdown (only if debate was run)
if result['route_type'] == 'debate':
    final_rec = result['full_result']
    print("Agent Breakdown:")
    print("-"*40)
    for agent_type, vote in final_rec.agent_breakdown.items():
        print(f"\n{agent_type.upper()}:")
        print(f"  Recommendation: {vote.recommendation.value}")
        print(f"  Confidence: {vote.confidence:.0%}")
        print(f"  Reasoning: {vote.reasoning[:200]}...")

## 5. Risk Tolerance Inference Examples

The orchestrator infers risk tolerance from natural language cues:

In [ ]:
# Test risk tolerance inference with different query styles

test_queries = [
    # Conservative queries
    "I'm nearing retirement and want a safe dividend stock. Is Maybank suitable for capital preservation?",
    
    # Moderate queries  
    "Looking for a balanced investment in CIMB with reasonable growth potential.",
    
    # Aggressive queries
    "I want maximum growth and don't mind volatility. Should I buy Public Bank for high returns?"
]

print("Risk Tolerance Inference Demo")
print("="*60)

for query in test_queries:
    print(f"\nQuery: {query[:80]}...")
    
    # Just classify without running full debate
    classification = orchestrator.classify_query(query)
    
    print(f"  → Risk Tolerance: {classification['risk_tolerance'].upper()}")
    print(f"  → Company: {classification.get('company', 'N/A')}")
    print(f"  → Needs Debate: {classification['needs_debate']}")
    print(f"  → Reasoning: {classification['reasoning'][:100]}...")

## 6. Single Agent Queries

Some queries only need a single specialist agent:

In [ ]:
# Example: Single agent query (sentiment-specific)
single_query = "What is the recent news sentiment for CIMB?"

print(f"Query: {single_query}")
print("-"*40)

# Classify first to show routing decision
classification = orchestrator.classify_query(single_query)
print(f"Classification:")
print(f"  Needs Debate: {classification['needs_debate']}")
print(f"  Agent: {classification['agent_type']}")
print(f"  Company: {classification.get('company')}")

# Route the query
result = orchestrator.route_query(single_query)

print(f"\nRoute Type: {result['route_type']}")
print(f"Agent Used: {result['agent_used']}")
print(f"\nResponse Preview:")
print(result['response'][:800] + "...")

In [ ]:
# Example: Valuation-specific query
valuation_query = "What is the volatility and Sharpe ratio for Maybank?"

print(f"Query: {valuation_query}")
print("-"*40)

result = orchestrator.route_query(valuation_query)

print(f"Route Type: {result['route_type']}")
print(f"Agent Used: {result['agent_used']}")
print(f"\nResponse Preview:")
print(result['response'][:800] + "...")

## 7. Conservative Investor Analysis

Run a full debate with explicit conservative risk tolerance:

In [ ]:
# Conservative investor query - explicit risk tolerance in language
conservative_query = "I'm a retiree looking for a safe, stable dividend stock. Should I invest in Maybank for capital preservation?"

print(f"Query: {conservative_query}")
print(f"\n{'='*60}")
print("Running analysis for CONSERVATIVE investor...")
print(f"{'='*60}\n")

# Route query - will infer conservative risk tolerance
result = orchestrator.route_query(conservative_query)

# Debug: Show classification details
print("Classification Details:")
classification = result.get('classification', {})
print(f"  Needs Debate: {classification.get('needs_debate')}")
print(f"  Agent Type: {classification.get('agent_type')}")
print(f"  Company: {classification.get('company')}")
print(f"  Reasoning: {classification.get('reasoning')}")

print(f"\nRoute Type: {result['route_type']}")
print(f"Inferred Risk Tolerance: {result['risk_tolerance'].upper()}")
print(f"Recommendation: {result.get('recommendation', 'N/A')}")

if result['route_type'] == 'debate':
    print(f"Confidence: {result['confidence']:.0%}")
    print(f"Consensus: {result['consensus']:.0%}")
elif result['route_type'] == 'single_agent':
    print(f"Agent Used: {result['agent_used']}")
    print(f"\nResponse Preview:\n{result['response'][:500]}...")
elif result['route_type'] == 'error':
    print(f"Error: {result['response']}")

In [ ]:
# Aggressive investor query
aggressive_query = "I want maximum growth and I'm willing to take high risks. Should I buy CIMB for aggressive returns?"

print(f"Query: {aggressive_query}")
print(f"\n{'='*60}")
print("Running analysis for AGGRESSIVE investor...")
print(f"{'='*60}\n")

# Route query - will infer aggressive risk tolerance
result = orchestrator.route_query(aggressive_query)

print(f"\nInferred Risk Tolerance: {result['risk_tolerance'].upper()}")
print(f"Recommendation: {result.get('recommendation', 'N/A')}")
if result['route_type'] == 'debate':
    print(f"Confidence: {result['confidence']:.0%}")
    print(f"Consensus: {result['consensus']:.0%}")

## 8. View Debate Transcript

In [ ]:
# Get debate transcript from the most recent debate
transcript = orchestrator.get_debate_transcript()
print(transcript)

In [ ]:
# Evaluate debate quality
evaluator = DebateEvaluator()

# Get debate data from orchestrator
debate_data = orchestrator.export_debate()

if debate_data:
    # Get the final recommendation from the last result
    final_rec_value = result.get('recommendation', 'HOLD')
    
    # Evaluate the debate
    metrics = evaluator.evaluate_debate(
        debate_state=debate_data,
        final_recommendation=final_rec_value
    )

    print("Debate Quality Metrics:")
    print("-"*40)
    print(f"Consensus Quality:     {metrics.consensus_quality:.1%}")
    print(f"Reasoning Consistency: {metrics.reasoning_consistency:.1%}")
    print(f"Convergence Rate:      {metrics.convergence_rate:.1%}")
    print(f"Recommendation Conf.:  {metrics.recommendation_confidence:.1%}")
    print(f"Rounds to Consensus:   {metrics.rounds_to_consensus}")
    print(f"Total Messages:        {metrics.total_messages}")
else:
    print("No debate data available. Run a debate first.")

## Summary

This notebook demonstrated:
- **Natural language query routing** via `orchestrator.route_query()`
- **Automatic risk tolerance inference** from query language (conservative/moderate/aggressive)
- **Query classification** - determining single agent vs multi-agent debate
- **Multi-agent debates** with three specialist agents
- **Single agent queries** for specific factual information
- **Debate evaluation** with quality metrics

### Key API Methods:
```python
# Main entry point - routes query automatically
result = orchestrator.route_query("Should I invest in Maybank?")

# Just classify without running (useful for testing)
classification = orchestrator.classify_query("Is CIMB a safe investment?")

# Direct debate (bypasses classification)
recommendation = orchestrator.run_debate("Maybank", risk_tolerance="conservative")
```

### Next Steps:
1. Run `streamlit run app.py` to launch the interactive UI
2. Try different query styles to see risk tolerance inference
3. Compare recommendations for same stock with different risk profiles